# Imports

## Libraries

In [3]:
import os
import re
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import gensim.downloader as api
word2vec_model = api.load("word2vec-google-news-300")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Data

### Raw

In [4]:
data_folder_path = '/text-mining/data/02_text_representation/Corpus-representacion'

In [5]:
def save_files_to_dict(folder_path):
    files_dict = {}
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                    files_dict[file_path] = f.read()
            except Exception as e:
                print(f"Could not read file {file_path}: {e}")
    return files_dict

In [6]:
data_dict = save_files_to_dict(data_folder_path)
data = list(data_dict.values())
len(data)

866

### Labels

In [13]:
categories = [path.split('/')[-2] for path in data_dict.keys()]
unique_categories = list(set(categories))
print(unique_categories)

['comp.sys.ibm.pc.hardware', 'sci.electronics', 'comp.sys.mac.hardware', 'rec.autos', 'talk.politics.mideast', 'rec.sport.hockey', 'talk.politics.guns']


### Representations

In [16]:
tf_representations = np.load("../02_text_representation/02_text_representations_tf.npy")
tfidf_representations = np.load("../02_text_representation/02_text_representations_tfidf.npy")
word2vec_avg_representations = np.load("../02_text_representation/02_text_representations_word2vec_avg.npy")
word2vec_sum_representations = np.load("../02_text_representation/02_text_representations_word2vec_sum.npy")

# Clustering

## Non-hierarchical
### K-means
## Hierarchical
### Agglomerative
### Divisive
## Topic modeling


# Evaluation
## Internal validation
### Silhouette score
### Calinski-Harabasz index
### Davies-Bouldin index
## External validation
### Confusion matrix
### Classification report
### Adjusted Rand index